In [16]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import requests
from geopy.geocoders import Nominatim
import geopy.distance
from sklearn.cluster import DBSCAN
import pickle



pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [26]:
df = pd.read_excel("/content/drive/MyDrive/Dataset/data2.xlsx")

In [27]:
df.head()

System Code Crash Date Crash Time              City     County  \
0  State Highway 2021-01-01   00:00:00  GLENWOOD SPRINGS   GARFIELD   
1    County Road 2021-01-01   00:05:00            DENVER   LA PLATA   
2    City Street 2021-01-01   00:17:00       WESTMINSTER  JEFFERSON   
3    City Street 2021-01-01   00:25:00            DENVER     DENVER   
4    County Road 2021-01-01   00:26:00            DENVER  JEFFERSON   

    Latitude   Longitude          Location 1 Location 2      Road Description  \
0  39.525490 -107.322530      SOUTH GLEN AVE    27TH ST  Intersection Related   
1  39.752829 -104.939516              CR 228       MM 4      Non-Intersection   
2  39.828850 -105.044020        TENNYSON ST   74TH AVE       Non-Intersection   
3  39.704610 -105.013560  2200 BLK W GILL PL  S ZUNI ST      Non-Intersection   
4  39.752829 -104.939516          W 32ND AVE  W 37TH DR  Intersection Related   

  Road Condition Lighting Conditions Weather Condition  Rush Hour  \
0          Snowy            Daylight              Snow          0   
1            Icy    Dark – Unlighted             Clear          0   
2            Dry    Dark – Unlighted             Clear          0   
3            Dry    Dark – Unlighted             Clear          0   
4            Dry    Dark – Unlighted             Clear          0   

  Alcohol Suspected Speed Limit Category Severity  Month  
0                No                  Low   slight      1  
1                No                  Low   slight      1  
2                No                  Low   slight      1  
3                No                  Low   slight      1  
4                No                  Low   slight      1

In [28]:
required_columns = ['Latitude', 'Longitude','Weather Condition', 'Rush Hour', 'Severity', ]
df = df[required_columns]
df.head()

Latitude   Longitude Weather Condition  Rush Hour Severity
0  39.525490 -107.322530              Snow          0   slight
1  39.752829 -104.939516             Clear          0   slight
2  39.828850 -105.044020             Clear          0   slight
3  39.704610 -105.013560             Clear          0   slight
4  39.752829 -104.939516             Clear          0   slight

In [29]:
df = df.dropna(subset=['Latitude', 'Longitude', 'Weather Condition', 'Rush Hour', 'Severity'])
df = df[(df['Latitude'].between(36.9, 41.0)) & (df['Longitude'].between(-109.1, -102.0))]

In [30]:
severity_mapping = {'slight': 0, 'serious': 1, 'fatal': 2}
df['Severity'] = df['Severity'].map(severity_mapping)

In [31]:
df.dtypes

Latitude             float64
Longitude            float64
Weather Condition     object
Rush Hour              int64
Severity               int64
dtype: object

In [32]:
print(df['Weather Condition'].unique())

['Snow' 'Clear' 'Sleet or Hail' 'Rain' 'Wind' 'Cloudy' 'Fog'
 'Blowing Snow' 'Freezing Rain or Freezing Drizzle' 'Dust']


In [33]:
geolocator = Nominatim(user_agent="colorado_road_risk_analysis")

In [34]:
# Function to filter data based on weather condition and rush hour
def filter_data(df, weather_condition, rush_hour):
    valid_weather_conditions = ['Snow', 'Clear', 'Sleet or Hail', 'Rain', 'Wind', 'Cloudy', 'Fog',
                                'Blowing Snow', 'Freezing Rain or Freezing Drizzle', 'Dust']
    if weather_condition not in valid_weather_conditions:
        raise ValueError(f"Invalid weather condition: {weather_condition}. Must be one of {valid_weather_conditions}")

    filtered_df = df[
        (df['Weather Condition'] == weather_condition) &
        (df['Rush Hour'] == (1 if rush_hour else 0))
    ]

    if len(filtered_df) < 5:
        print(f"Warning: Only {len(filtered_df)} crashes match the conditions. DBSCAN may not identify meaningful clusters.")
        if len(filtered_df) == 0:
            print("No crashes match the specified conditions. Exiting.")
            return None

    print(f"Filtered dataset: {len(filtered_df)} rows")
    return filtered_df

In [35]:
def get_coordinates(location):
    try:
        location_data = geolocator.geocode(location + ", Colorado, USA")
        if location_data:
            return location_data.latitude, location_data.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Geocoding error: {e}")
        return None, None

In [36]:
def get_route_coordinates(start_lat, start_lon, end_lat, end_lon):
    api_key = "5b3ce3597851110001cf6248933ad121d64d415f9c9a132f93c980aa"
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={start_lon},{start_lat}&end={end_lon},{end_lat}"
    try:
        response = requests.get(url)
        data = response.json()
        coordinates = data['features'][0]['geometry']['coordinates']
        return [(lat, lon) for lon, lat in coordinates]
    except Exception as e:
        print(f"Error fetching route: {e}")
        return [(start_lat, start_lon), (end_lat, end_lon)]

In [37]:
def sample_points_every_mile(coordinates):
    if not coordinates or len(coordinates) < 2:
        return coordinates
    total_distance = 0
    distances = []
    for i in range(len(coordinates) - 1):
        start = coordinates[i]
        end = coordinates[i + 1]
        distance = geopy.distance.distance(start, end).miles
        total_distance += distance
        distances.append(distance)
    if total_distance < 1:
        return coordinates
    num_points = int(total_distance) + 2
    sampled_points = [coordinates[0]]
    current_distance = 0
    target_distance = 1.0
    i = 0
    while i < len(coordinates) - 1:
        current_distance += distances[i]
        if current_distance >= target_distance:
            overshoot = current_distance - target_distance
            fraction = (distances[i] - overshoot) / distances[i]
            lat1, lon1 = coordinates[i]
            lat2, lon2 = coordinates[i + 1]
            interpolated_lat = lat1 + (lat2 - lat1) * fraction
            interpolated_lon = lon1 + (lon2 - lon1) * fraction
            sampled_points.append((interpolated_lat, interpolated_lon))
            target_distance += 1.0
        i += 1
    if sampled_points[-1] != coordinates[-1]:
        sampled_points.append(coordinates[-1])
    return sampled_points

In [38]:
def dbscan_hotspots(sampled_points, crash_data, eps_miles=0.5, min_samples=5):
    eps = eps_miles / 69.0
    coords = crash_data[['Latitude', 'Longitude']].values
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    crash_data['Cluster'] = db.labels_
    hotspots = []
    for i, (lat, lon) in enumerate(sampled_points, 1):
        score = 0
        crash_count = 0
        for idx, crash in crash_data.iterrows():
            if crash['Cluster'] == -1:
                continue
            crash_lat = crash['Latitude']
            crash_lon = crash['Longitude']
            distance = geopy.distance.distance((lat, lon), (crash_lat, crash_lon)).miles
            if distance <= eps_miles:
                score += (crash['Severity'] + 1)
                crash_count += 1
        hotspots.append({
            'point': f"Point {i}",
            'latitude': lat,
            'longitude': lon,
            'score': score,
            'crash_count': crash_count
        })
    return hotspots

In [39]:
def print_top_hotspots(hotspots, top_n=3):
    top_hotspots = sorted(hotspots, key=lambda x: x['score'], reverse=True)[:top_n]
    for hotspot in top_hotspots:
        print(f"{hotspot['point']} (Lat: {hotspot['latitude']:.4f}, Lon: {hotspot['longitude']:.4f}): "
              f"Score = {hotspot['score']:.4f}, Crashes = {hotspot['crash_count']}")

In [40]:
def visualize_hotspots(sampled_points, crash_data, hotspots, method_name, weather_condition, rush_hour):
    plt.figure(figsize=(12, 8))
    plt.scatter(crash_data['Longitude'], crash_data['Latitude'], c='gray', alpha=0.3, s=10, label='Crashes')
    route_lats = [p[0] for p in sampled_points]
    route_lons = [p[1] for p in sampled_points]
    plt.plot(route_lons, route_lats, 'b-', label='Route')
    top_hotspots = sorted(hotspots, key=lambda x: x['score'], reverse=True)[:3]
    for hotspot in top_hotspots:
        plt.scatter(hotspot['longitude'], hotspot['latitude'], c='g', s=100,
                    marker='*', label=f'{method_name} Hotspot' if hotspot == top_hotspots[0] else "")
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    rush_hour_str = "Rush Hour" if rush_hour else "Non-Rush Hour"
    plt.title(f'Hotspot Detection Along Route ({method_name})\nWeather: {weather_condition}, Time: {rush_hour_str}')
    plt.legend()
    plt.grid(True)
    plt.savefig('hotspot_visualization_dbscan_conditions.png')
    plt.close()

In [41]:
def save_configuration(method_name, params, filename="best_hotspot_method_dbscan_conditions.pkl"):
    with open(filename, 'wb') as file:
        pickle.dump({
            'method_name': method_name,
            'params': params
        }, file)
    print(f"DBSCAN configuration saved as '{filename}'")

In [42]:
def main(source, destination, weather_condition, rush_hour):
    # Step 1: Filter data based on weather condition and rush hour
    filtered_df = filter_data(df, weather_condition, rush_hour)
    if filtered_df is None:
        return

    # Step 2: Get route coordinates
    source_lat, source_lon = get_coordinates(source)
    dest_lat, dest_lon = get_coordinates(destination)
    if source_lat is None or dest_lat is None:
        print("Error: Could not geocode source or destination.")
        return

    route_coords = get_route_coordinates(source_lat, source_lon, dest_lat, dest_lon)
    if not route_coords:
        print("Error: Could not fetch route.")
        return

    # Step 3: Sample points every mile
    sampled_points = sample_points_every_mile(route_coords)
    print(f"Sampled {len(sampled_points)} points along the route.")

    # Step 4: Apply DBSCAN hotspot detection
    print("\n=== Running DBSCAN ===")
    hotspots = dbscan_hotspots(sampled_points, filtered_df.copy())
    print_top_hotspots(hotspots)

    # Step 6: Save the DBSCAN configuration
    method_name = "DBSCAN"
    params = {
        'eps_miles': 0.5,
        'min_samples': 5,
        'weather_condition': weather_condition,
        'rush_hour': rush_hour
    }
    save_configuration(method_name, params)

    # Step 7: Visualize the results
    visualize_hotspots(sampled_points, filtered_df, hotspots, method_name, weather_condition, rush_hour)
    print("Visualization saved as 'hotspot_visualization_dbscan_conditions.png'")

In [43]:
if __name__ == "__main__":
    source = "Denver"
    destination = "Boulder"
    weather_condition = "Rain"  # Use one of: 'Snow', 'Clear', 'Sleet or Hail', 'Rain', 'Wind', 'Cloudy', 'Fog', 'Blowing Snow', 'Freezing Rain or Freezing Drizzle', 'Dust'
    rush_hour = True
    main(source, destination, weather_condition, rush_hour)

Filtered dataset: 5084 rows
Sampled 30 points along the route.

=== Running DBSCAN ===
Point 29 (Lat: 40.0147, Lon: -105.2597): Score = 37.0000, Crashes = 36
Point 1 (Lat: 39.7392, Lon: -104.9855): Score = 33.0000, Crashes = 33
Point 2 (Lat: 39.7476, Lon: -104.9855): Score = 23.0000, Crashes = 22
DBSCAN configuration saved as 'best_hotspot_method_dbscan_conditions.pkl'
Visualization saved as 'hotspot_visualization_dbscan_conditions.png'
